In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd
import numpy as np
import math
import os
import unicodedata
import string
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
import datetime
from dateutil.parser import parse

In [2]:
# WINDOWS
chromedriver_path = "INSERT HERE"
user_data_path = "INSERT HERE"

# MAC
chromedriver_path = "INSERT HERE"
user_data_path = "INSERT HERE"

# OPTIONS
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("disable-dev-shm-usage")
options.add_argument("headless")
options.add_argument(f"user-data-dir={user_data_path}")
# driver = webdriver.Chrome(f"{chromedriver_path}", chrome_options=options)
# driver.get('https://pro.imdb.com')


# driver.get("https://pro.imdb.com/title/tt0076759/boxoffice")

# src = driver.page_source
# # driver.quit()
# parser = BeautifulSoup(src, 'lxml')


In [3]:
import nbimporter
import ast
from create_dataset import createDataset

df = createDataset(9)
one,two,three,four,five,six,seven,eight,nine = df[:1000], df[1000:2000],df[2000:3000],df[3000:4000],df[4000:5000],df[5000:6000],df[6000:7000],df[7000:8000],df[8000:]

Importing Jupyter notebook from create_dataset.ipynb
Reading data....


Complete!


In [4]:
df['date'] = pd.to_datetime(df['date'])
df['Week Day'] = df['date'].apply(lambda x: x.weekday())

In [5]:
df[['Week Day', 'date', 'name']]

days={
  'monday': 0,
  'tuesday': 1,
  'wednesday': 2,
  'thursday': 3,
  'friday': 4,
  'saturday': 5,
  'sunday': 6,
}

df.loc[(df['Week Day']<4)][['name', 'Week Day', 'date']].head()

,name,Week Day,date
8,The Cherry Orchard,2,2000-01-12
9,Next Friday,2,2000-01-12
17,Le pique-nique de Lulu Kreutz,2,2000-01-19
18,The Suburbans,2,2000-01-19
29,Punks,1,2000-01-25


In [6]:
def retry(ExceptionToCheck, tries=20, delay=3, backoff=2, logger=None):
  """
    Modified from source: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    Objective
    ----------
    Exponential backoff function.

    Parameters
    ----------
    ExceptionToCheck : Exception or tuple
      the exception to check. may be a tuple of exceptions to check
      Possible values:
        ConnectionResetError, (TimeoutError, ConnectionError)
    tries : int
      number of times to try (not retry) before giving up
    delay : int
      initial delay between retries in seconds
    backoff: int
      backoff multiplier
      E.g. value of 2 will double the delay each retry
    logger : logging.Logger instance
      logger to use
  """
  def deco_retry(f):
    @wraps(f)
    def f_retry(*args, **kwargs):
      mtries, mdelay = tries, delay
      while mtries > 1:
        try:
          return f(*args, **kwargs)
        except ExceptionToCheck:
          msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
          if logger:
            #logger.exception(msg) # would print stack trace
            logger.warning(msg)
          else:
            print(msg)
          time.sleep(mdelay)
          mtries -= 1
          mdelay *= backoff
        return f(*args, **kwargs)
      return f_retry  # true decorator
  return deco_retry  

In [8]:
def daily_theatres(parser, datezor):
  # GRAB DATA FROM OPENING 7 DAYS
  from datetime import date
  parent = parser.find('td', attrs = {'data-sort-value':str(datezor)}).parent.parent
  children = parent.find_all('tr')

  movie_datetime = date(int(datezor[0:4]), int(datezor[4:6]), int(datezor[6:8]))

  result_object = {}
  result_object['dates'] = []
  result_object['theatres'] =[]
  result_object['gross'] = []
  result_object['gross_to_date'] = []

  dates = []
  theatres = []
  gross = []
  gross_to_date = []

  #define headers
  headers = children[0].find_all('th')

  for idx, header in enumerate(headers):
    if header.text.strip()=='Date':
      date_col = idx
    elif header.text.strip()=='Single Day Gross':
      result_object['value']='day'
      gross_col = idx
    elif header.text.strip()=='Weekend Gross':
      result_object['value']='week'
      gross_col = idx
    elif header.text.strip()=='Gross':
      result_object['value']='week'
      gross_col = idx
    elif header.text.strip()=='Theaters':
      theaters_col = idx
    elif header.text.strip()=='Screens':
      theaters_col = idx
    elif header.text.strip()=='Gross Since Release':
      result_object['value']='day'
      gtd_col = idx
    elif header.text.strip()=='Gross to Date':
      result_object['value']='week'
      gtd_col = idx


  target_children = []
  for child in children[1:]:
      current_date = child.td['data-sort-value']
      current_datetime = date(int(current_date[0:4]), int(current_date[4:6]), int(current_date[6:8]))
      difference = (current_datetime-movie_datetime).days
      if type(current_date)==str and difference >=0 and difference < 7:
        target_children.append(child)

  for child in target_children:
    try:
      current_date = child.td['data-sort-value']
      current_datetime = date(int(current_date[0:4]), int(current_date[4:6]), int(current_date[6:8]))
      difference = (current_datetime-movie_datetime).days
      if type(current_date)==str and difference >=0 and difference < 7:
        try:
          current_theatres = child.find_all('td')[theaters_col]['data-sort-value']
        except:
          current_theatres = np.nan
        try:
          if result_object['value']=='day':
            current_gross = child.find_all('td')[gross_col].div.p.text.strip()
          else:
            current_gross = child.find_all('td')[gross_col].div.text.strip()
        except:
          current_gross = np.nan
        try:
          current_gtd = child.find_all('td')[gtd_col].div.text.strip()
        except:
          current_gtd = np.nan

        print(Fore.GREEN + f"Date: {current_date}")
        print(Fore.GREEN + f"Theatres: {current_theatres}")
        print(Fore.GREEN + f"Gross: {current_gross}")
        print(Fore.GREEN + f"Gross to date: {current_gtd}")
        result_object['dates'].append(current_date)
        result_object['theatres'].append(current_theatres)
        result_object['gross'].append(current_gross)
        result_object['gross_to_date'].append(current_gtd)
    except:
      continue

  print(result_object)
  # driver.quit()

  return result_object

In [9]:
def scrape(df, thread_count, new_df=None):
  if type(new_df)!=pd.DataFrame:
    movies = pd.DataFrame()
  else:
    movies = new_df
  counter=0
  for row in df.itertuples():
    print(f"{counter}/{len(df)}")

    counter +=1
    profile = row.profile
    name = row.name
#     release = row.release
    release = row.date
    '''
        GET OPENING THEATRE COUNT
    '''
    try:
      # START UP WEB DRIVER
      driver = webdriver.Chrome(chromedriver_path, chrome_options=options)
      @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
      def get_prop():
          driver.get("https://pro.imdb.com" + profile + "boxoffice")
      get_prop()
      src = driver.page_source
      parser = BeautifulSoup(src, 'lxml')
      # GRAB OPENING DAY THEATRE COUNT
      try:
        release_date = parser.find_all('div', class_='a-fixed-right-grid-col a-col-right')[1].find('div', class_="a-row a-spacing-base").span.a.text
        release_date = release_date.replace(",", "").split("\n")
        #CREATE MONTHS DICTIONARY
        months = dict(Jan=1, Feb=2, Mar=3, Apr=4, May=5, Jun=6, Jul=7, Aug=8, Sep=9, Oct=10, Nov=11, Dec=12)
        #CALCULATE YEAR
        year = str(release_date[2])
        #CALCULATE MONTH
        month = str(release_date[0])
        month = str(months[month])
        if len(month) > 1:
          pass
        else:
          month = '0' + str(month)
        #CALCULATE DAY
        if int(release_date[1]) < 10:
          day = '0' + str(release_date[1])
        else:
          day = str(release_date[1])
        date = year + month + day
        
        try:
          '''
            DEFINE TABLE HEADERS
          '''
          parent = parser.find('td', attrs = {'data-sort-value':str(date)}).parent.parent
          children = parent.find_all('tr')


          #define headers
          headers = children[0].find_all('th')
  #         day, week = False, False
          for idx, header in enumerate(headers):
            if header.text.strip()=='Date':
              date_col = idx
            elif header.text.strip()=='Single Day Gross':
          #     result_object['value']=='day'
              gross_col = idx
            elif header.text.strip()=='Weekend Gross':
          #     result_object['value']=='week'
              gross_col = idx
            elif header.text.strip()=='Theaters':
              theaters_col = idx
            elif header.text.strip()=='Gross Since Release':
          #     result_object['value']=='day'
              gtd_col = idx
            elif header.text.strip()=='Gross to Date':
          #     result_object['value']=='week'
              gtd_col = idx


          # GRAB DATA FROM OPENING 7 DAYS
          opening_week_data = daily_theatres(parser, date)
        except:
          opening_week_data = np.nan

        try:
          opening_day = int(parser.find('td', attrs = {'data-sort-value':str(date)}).parent.find_all('td')[theaters_col]['data-sort-value'])
        except:
          opening_day = np.nan
          

        #find weekday of date
        weekday =  datetime.datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d').weekday()
        if weekday>3:
          try:
            opening_weekend = int(parser.find('td', attrs = {'data-sort-value':str(date)}).parent.find_all('td')[theaters_col]['data-sort-value'])
          except:
            opening_weekend = np.nan
        else:
          #TRY FRIDAY SATURDAY OR SUNDAY (incase theatres are closed on holiday)
          def returnTheatres(year, month, day, day_of_week, weekday):
            difference = day_of_week - int(weekday)
            opening_date = datetime.datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')
            result = opening_date + datetime.timedelta(days=difference)
            day = str(result.day)
            month = str(result.month)
            year = str(result.year)
            if len(month) > 1:
              pass
            else:
              month = '0' + str(month)
            #CALCULATE DAY
            if int(day) < 10:
              day = '0' + str(day)
            else:
              day = str(day)
            date = f"{year}{month}{day}"
            opening_weekend = parser.find('td', attrs = {'data-sort-value':str(date)}).parent.find_all('td')[theaters_col]['data-sort-value']
            return int(opening_weekend)

          try:
            opening_weekend = returnTheatres(year, month, day, 4, weekday)
          except:
            try:
              opening_weekend = returnTheatres(year, month, day, 5, weekday)
            except:
              try:
                opening_weekend = returnTheaters(year, month, day, 6, weekday)
              except:
                opening_weekend = np.nan

        if np.isnan(opening_day):
          print(Fore.RED + "Opening Day")
        else:
          print(Fore.GREEN + "Opening Day")

        if np.isnan(opening_weekend):
          print(Fore.RED + "Opening Weekend")
        else:
          print(Fore.GREEN + "Opening Weekend")

      except Exception as e:
        print("Error: ", e)
        opening_day = np.nan
        opening_weekend = np.nan
        print(Fore.RED + "Opening Day")
        print(Fore.RED + "Opening Weekend")

      # GATHER PRE RELEASE SCREENING BOOLEAN
      try:
        pre_release_theatres = parser.find('table', {"id": "box_office_mojo"}).find_all('tr')[1].find_all('td')[0]['data-sort-value']
        first_date = str(pre_release_theatres)[:4] + "-" + str(pre_release_theatres)[4:6] + "-" + str(pre_release_theatres)[6:8]
        print(first_date)
        pre_release_theatres = first_date
        print(Fore.GREEN + f"Pre-release Screening: {pre_release_theatres}")
      except:
        pre_release_theatres = np.nan
        print(Fore.RED + "Pre-release Screening")


#       # GATHER MAX THEATRE BOOLEAN
#       try:
#         theatre_counts = []
#         elements = parser.find('table', {"id": "box_office_mojo"}).find_all('tr')
#         for element in elements:
#           try:
#             theatre_counts.append(int(element.find_all('td')[4].text.strip().replace(",", "")))
#           except:
#             continue
#         max_theatres = max(theatre_counts)
#         print(Fore.GREEN + f"Max Theatres: {max_theatres}")
#       except:
#         max_theatres = np.nan
#         print(Fore.RED + "Max Theatres")

      try:
        @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
        def get_prop():
            driver.get("https://pro.imdb.com" + profile + "cast")
        get_prop()
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        contacts = parser.find('div', attrs = {'id':'contacts'}).find_all('div', class_='header_section')
        for contact in contacts:
          result = contact.find_next('div', class_='contacts_header').text.strip()
          if result == 'Distributor':
            distributor = contact.a.text.strip()
        print(Fore.GREEN + f'Distributor: {distributor}')
        clear_output(wait = True)
      except Exception as e:
        distributor = np.nan
        print(f"Error: {e}")
        print(Fore.RED + 'Distributor')
        clear_output(wait = True)
        
        
      
      driver.quit()
      movie = pd.DataFrame({
        'name': [name],
        'opening_day_theatres': [opening_day],
        'opening_week_data': [opening_week_data],
        'opening_weekend': [opening_weekend],
        'pre_release': [pre_release_theatres],
        'release': [release],
        'profile': [profile],
        'distributor': [distributor],
        'error': [np.nan]
      })
    except Exception as e:
      print(f"Error: {e}")
      movie = pd.DataFrame({
        'name': [name],
        'opening_day_theatres': [np.nan],
        'opening_week_data': [np.nan],
        'opening_weekend': [np.nan],
        'pre_release': [np.nan],
        'release': [release],
        'profile': [profile],
        'distributor': [np.nan],
        'error': [e]
      })
    movies = movies.append(movie, ignore_index=True)
  movies.to_csv(f"/Volumes/128/data/{thread_count}.csv", index=False)

In [10]:
scrape(one, 'one')

999/1000
Date: 20020823
Theatres: 2154
Gross: $2,042,344
Gross to date: $2,042,344
Date: 20020824
Theatres: 2154
Gross: $2,276,895
Gross to date: $4,319,239
Date: 20020825
Theatres: 2154
Gross: $1,438,997
Gross to date: $5,758,236
Date: 20020826
Theatres: 2154
Gross: $495,204
Gross to date: $6,253,440
Date: 20020827
Theatres: 2154
Gross: $508,362
Gross to date: $6,761,802
Date: 20020828
Theatres: 2154
Gross: $429,464
Gross to date: $7,191,266
Date: 20020829
Theatres: 2154
Gross: $372,426
Gross to date: $7,563,692
{'dates': ['20020823', '20020824', '20020825', '20020826', '20020827', '20020828', '20020829'], 'theatres': ['2154', '2154', '2154', '2154', '2154', '2154', '2154'], 'gross': ['$2,042,344', '$2,276,895', '$1,438,997', '$495,204', '$508,362', '$429,464', '$372,426'], 'gross_to_date': ['$2,042,344', '$4,319,239', '$5,758,236', '$6,253,440', '$6,761,802', '$7,191,266', '$7,563,692'], 'value': 'day'}
Opening Day
Opening Weekend
2002-08-23
Pre-release Screening: 2002-08-23
Distribu

In [56]:
type(two)==pd.DataFrame

True

In [10]:
scrape(two[200:], 'two', pd.read_csv("/Volumes/128/data/two.csv")[:200])

799/800
Date: 20050211
Theatres: 3575
Gross: $13,966,468
Gross to date: $13,966,468
Date: 20050212
Theatres: 3575
Gross: $18,581,456
Gross to date: $32,547,924
Date: 20050213
Theatres: 3575
Gross: $10,594,290
Gross to date: $43,142,214
Date: 20050214
Theatres: 3575
Gross: $7,515,442
Gross to date: $50,657,656
Date: 20050215
Theatres: 3575
Gross: $2,658,085
Gross to date: $53,315,741
Date: 20050216
Theatres: 3575
Gross: $2,346,532
Gross to date: $55,662,273
Date: 20050217
Theatres: 3575
Gross: $2,652,946
Gross to date: $58,315,219
{'dates': ['20050211', '20050212', '20050213', '20050214', '20050215', '20050216', '20050217'], 'theatres': ['3575', '3575', '3575', '3575', '3575', '3575', '3575'], 'gross': ['$13,966,468', '$18,581,456', '$10,594,290', '$7,515,442', '$2,658,085', '$2,346,532', '$2,652,946'], 'gross_to_date': ['$13,966,468', '$32,547,924', '$43,142,214', '$50,657,656', '$53,315,741', '$55,662,273', '$58,315,219'], 'value': 'day'}
Opening Day
Opening Weekend
2005-02-11
Pre-rel

In [11]:
scrape(three, 'three')

999/1000
Opening Day
Opening Weekend
2006-09-01
Pre-release Screening: 2006-09-01
Distributor: Goodbye Cruel Releasing


In [12]:
scrape(four, 'four')

999/1000
Date: 20090814
Theatres: 2988
Gross: $7,720,103
Gross to date: $7,720,103
Date: 20090815
Theatres: 2988
Gross: $6,374,563
Gross to date: $14,094,666
Date: 20090816
Theatres: 2988
Gross: $4,528,505
Gross to date: $18,623,171
Date: 20090817
Theatres: 2988
Gross: $2,338,419
Gross to date: $20,961,590
Date: 20090818
Theatres: 2988
Gross: $2,478,724
Gross to date: $23,440,314
Date: 20090819
Theatres: 2988
Gross: $2,055,357
Gross to date: $25,495,671
Date: 20090820
Theatres: 2988
Gross: $1,927,578
Gross to date: $27,423,249
{'dates': ['20090814', '20090815', '20090816', '20090817', '20090818', '20090819', '20090820'], 'theatres': ['2988', '2988', '2988', '2988', '2988', '2988', '2988'], 'gross': ['$7,720,103', '$6,374,563', '$4,528,505', '$2,338,419', '$2,478,724', '$2,055,357', '$1,927,578'], 'gross_to_date': ['$7,720,103', '$14,094,666', '$18,623,171', '$20,961,590', '$23,440,314', '$25,495,671', '$27,423,249'], 'value': 'day'}
Opening Day
Opening Weekend
2009-08-14
Pre-release Sc

In [13]:
scrape(five, 'five')

999/1000
Opening Day
Opening Weekend
Pre-release Screening
Distributor: Cohen Media Group


In [14]:
scrape(six, 'six')

999/1000
Date: 20140110
Theatres: 2
Gross: $9,280
Gross to date: $9,280
{'dates': ['20140110'], 'theatres': ['2'], 'gross': ['$9,280'], 'gross_to_date': ['$9,280'], 'value': 'week'}
Opening Day
Opening Weekend
2014-01-10
Pre-release Screening: 2014-01-10
Distributor: Big World Pictures


In [15]:
scrape(seven, 'seven')

999/1000
Opening Day
Opening Weekend
2012-08-31
Pre-release Screening: 2012-08-31
Distributor: Millennium Entertainment


In [16]:
scrape(eight, 'eight')

999/1000
Date: 20180209
Theatres: 3042
Gross: $3,736,369
Gross to date: $3,736,369
Date: 20180210
Theatres: 3042
Gross: $5,436,962
Gross to date: $9,173,331
Date: 20180211
Theatres: 3042
Gross: $3,380,955
Gross to date: $12,554,286
Date: 20180212
Theatres: 3042
Gross: $1,078,247
Gross to date: $13,632,533
Date: 20180213
Theatres: 3042
Gross: $1,517,713
Gross to date: $15,150,246
Date: 20180214
Theatres: 3042
Gross: $1,702,360
Gross to date: $16,852,606
Date: 20180215
Theatres: 3042
Gross: $895,111
Gross to date: $17,747,717
{'dates': ['20180209', '20180210', '20180211', '20180212', '20180213', '20180214', '20180215'], 'theatres': ['3042', '3042', '3042', '3042', '3042', '3042', '3042'], 'gross': ['$3,736,369', '$5,436,962', '$3,380,955', '$1,078,247', '$1,517,713', '$1,702,360', '$895,111'], 'gross_to_date': ['$3,736,369', '$9,173,331', '$12,554,286', '$13,632,533', '$15,150,246', '$16,852,606', '$17,747,717'], 'value': 'day'}
Opening Day
Opening Weekend
2018-02-09
Pre-release Screenin

In [17]:
scrape(nine, 'nine')

769/770
Opening Day
Opening Weekend
2019-09-22
Pre-release Screening: 2019-09-22
Error: 'NoneType' object has no attribute 'find_all'
Distributor


In [23]:
# threads = pd.DataFrame()
# thread_one = pd.read_csv("../../data/thread_one.csv")
# thread_two = pd.read_csv("../../data/thread_two.csv")
# threads = threads.append(thread_one, ignore_index=True)
# threads = threads.append(thread_two, ignore_index=True)

# threads.dropna(subset=['box_office'], inplace=True)
# threads = threads.reset_index(drop=True)
# threads = threads.drop_duplicates()
# print(len(threads))
# threads = threads[7000:]


# one = threads[:1000]
# two = threads[1000:2000]
# three = threads[2000:3000]
# four = threads[3000:4000]
# five = threads[4000:5000]
# six = threads[5000:6000]
# seven = threads[6000:len(threads)]

13947


In [24]:
scrape(one, 'eight')

Error:  list index out of range
Opening Day
Opening Weekend
1990-11-02
Pre-release Screening: 1990-11-02
Max Theatres: 2
Distributor: The Samuel Goldwyn Company


In [25]:
scrape(two, 'nine')

Opening Day
Opening Weekend
2005-04-22
Pre-release Screening: 2005-04-22
Max Theatres: 64
Distributor: IFC Films


In [26]:
scrape(three, 'ten')

Opening Day
Opening Weekend
Pre-release Screening
Max Theatres
Distributor: Columbia Pictures


In [27]:
scrape(four, 'eleven')

Error:  list index out of range
Opening Day
Opening Weekend
2008-12-26
Pre-release Screening: 2008-12-26
Max Theatres: 3
Distributor: IFC Films


In [28]:
scrape(five, 'twelve')

Opening Day
Opening Weekend
2015-02-20
Pre-release Screening: 2015-02-20
Max Theatres: 9
Distributor: BBC Worldwide Americas


In [29]:
scrape(six, 'thirteen')

Opening Day
Opening Weekend
1997-10-03
Pre-release Screening: 1997-10-03
Max Theatres: 12
Distributor: Orion Pictures


In [30]:
scrape(seven, 'fourteen')

Opening Day
Opening Weekend
2019-11-08
Pre-release Screening: 2019-11-08
Max Theatres: 88
Distributor: Well Go USA Entertainment
